In [163]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import os

In [164]:
if not os.path.isdir('Properties_philly_Kraggle_v2.csv'):
  print(os.listdir("/content"))

['.config', 'Properties_philly_Kraggle_v2.csv', 'sample_data']


In [ ]:
# philly_properties = np.genfromtxt('Properties_philly_Kraggle_v2.csv', delimiter=',')

In [ ]:
import pandas as pd
# df=pd.read_csv('Properties_philly_Kraggle_v2.csv', sep=',')
df=pd.read_csv('Properties_philly_Kraggle_v2.csv', sep=',', index_col=False)
df.dropna(inplace=True)
df.reset_index()
df.values

In [ ]:
df[-5:]
# TODO: Fix index after dropna

In [167]:
df.drop(['Address', 'Opening Bid', 'Book/Writ', 'OPA', 'Ward', 'Sheriff Cost', 'Advertising', 'Other', 'Record Deed', 'Zillow Estimate', 'Sale Date'], axis=1, inplace=True)

In [168]:
df.rename(columns={'finished \n(SqFt)':'SqFt', 'Sale Price/bid price': 'Sale Price', 'Zillow Address': 'Address'}, inplace=True)

In [ ]:
df.columns

## Analyze Sale Prices
And classify data into quantiles

In [ ]:
prices = df['Sale Price']
# prices.values
prices = prices.to_numpy()
prices_int = []
for price in prices:
  prices_int.append(int(price[1:].replace(',', '')))
prices = np.array(prices_int)
prices[1]

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [17.50, 13.50]
plt.rcParams["figure.autolayout"] = True

prices_index = np.array(list(range(prices.size)))

plt.title("Graph of Sale Price Data")
plt.plot(prices_index, prices, color="green")

plt.show()

In [ ]:
quartile_0 = min(prices) 
quartile_1 = np.quantile(prices, .25)
quartile_2 = np.quantile(prices, .50)
quartile_3 = np.quantile(prices, .75)
quartile_4 = max(prices)
print(quartile_0, quartile_1, quartile_2, quartile_3, quartile_4)

In [ ]:
Sales_price_category = []
for price in prices:
  if price < quartile_1:
    Sales_price_category.append(float(0))
  elif price < quartile_2:
    Sales_price_category.append(float(1))
  elif price < quartile_3:
    Sales_price_category.append(float(2))
  else:
    Sales_price_category.append(float(3))
Sales_price_category[-10:]

In [ ]:
df['Sales Price Category'] = Sales_price_category
df.drop(columns=['Sale Price'], inplace=True)
df[-10:]

## Pre process data for the Nerual Network

In [176]:
df.drop(columns=['Address', 'Attorney', 'Seller', 'Buyer'], inplace=True)

In [177]:
PropType = df['PropType']
prop_types = set()
for prop in PropType:
  prop_types.add(prop)
prop_types = list(prop_types)
new_prop = list()
for prop in PropType:
  new_prop.append(float(prop_types.index(prop)))
df['PropType'] = new_prop

In [178]:
df = df[df[' bathrooms '] != ' -   ']
df['bathrooms'] = df[' bathrooms ']
df.drop(columns=[' bathrooms '], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [179]:
df = df[df[' bedrooms '] != ' -   ']
df['bedrooms'] = df[' bedrooms ']
df.drop(columns=[' bedrooms '], inplace=True)

In [180]:
df = df[df[' Avg Walk&Transit score  '] != ' -   ']
df['Avg Walk&Transit score'] = df[' Avg Walk&Transit score  ']
df.drop(columns=[' Avg Walk&Transit score  '], inplace=True)

In [181]:
df = df[df[' Violent Crime Rate '] != ' -   ']
df['Violent Crime Rate'] = df[' Violent Crime Rate ']
df.drop(columns=[' Violent Crime Rate '], inplace=True)

In [182]:
df = df[df[' School Score  '] != ' -   ']
df['School Score'] = df[' School Score  ']
df.drop(columns=[' School Score  '], inplace=True)

In [183]:
df = df.astype({'bathrooms': float, 'bedrooms': float})

In [ ]:
# PropType = df['Sale Date']
# prop_types = set()
# for prop in PropType:
#   prop_types.add(prop)
# prop_types
# df.columns


In [184]:
for column_name in ['Rent Estimate', 'taxAssessment', 'Average comps']:
  new_col = list()
  for cell in df[column_name]:
    new_col.append(float(cell.replace(',', '')))
  df[column_name] = new_col

In [186]:
df.dtypes

Postal Code               float64
Water                     float64
PGW                       float64
Rent Estimate             float64
taxAssessment             float64
yearBuilt                 float64
SqFt                      float64
PropType                  float64
Average comps             float64
Sales Price Category        int64
bathrooms                 float64
bedrooms                  float64
Avg Walk&Transit score    float64
Violent Crime Rate        float64
School Score              float64
dtype: object

In [187]:
df.head(2)

,Postal Code,Water,PGW,Rent Estimate,taxAssessment,yearBuilt,SqFt,PropType,Average comps,Sales Price Category,bathrooms,bedrooms,Avg Walk&Transit score,Violent Crime Rate,School Score
0,19125.0,361.49,415.28,1500.0,174000.0,1921.0,898.0,0.0,240254.55,0,1.0,2.0,82.25,1.13,15.87
1,19136.0,323.76,0.00,1200.0,119700.0,1959.0,1120.0,2.0,150937.50,0,1.5,3.0,65.25,0.35,32.53


In [188]:
df = df.sample(frac = 1)
train_samples = df[:500].drop(columns=['Sales Price Category'])
train_labels = df[:500]['Sales Price Category']
test_samples = df[500:].drop(columns=['Sales Price Category'])
test_labels = df[500:]['Sales Price Category']

In [ ]:
train_samples

In [192]:
train_samples = train_samples.to_numpy()
train_labels = train_labels.to_numpy()
test_samples = test_samples.to_numpy()
test_labels = test_labels.to_numpy()

In [193]:
train_samples.dtype

dtype('float64')

## Building the neural network model

In [215]:
model = None

In [216]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [217]:
model = Sequential([
    Dense(units=16, input_shape=(14,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=4, activation='softmax')
])

In [218]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 16)                240       
                                                                 
 dense_16 (Dense)            (None, 32)                544       
                                                                 
 dense_17 (Dense)            (None, 4)                 132       
                                                                 
Total params: 916
Trainable params: 916
Non-trainable params: 0
_________________________________________________________________


In [219]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)

In [220]:
model.fit(x=train_samples,
          y=train_labels, validation_split=0.1,
          batch_size=10,
          epochs=200,
          shuffle=True,
          verbose=2
)

Epoch 1/200
45/45 - 1s - loss: 8559.1641 - accuracy: 0.2178 - val_loss: 7867.2671 - val_accuracy: 0.1800 - 916ms/epoch - 20ms/step
Epoch 2/200
45/45 - 0s - loss: 6867.6445 - accuracy: 0.2533 - val_loss: 6159.8770 - val_accuracy: 0.1800 - 124ms/epoch - 3ms/step
Epoch 3/200
45/45 - 0s - loss: 5320.4707 - accuracy: 0.2667 - val_loss: 4686.7627 - val_accuracy: 0.2200 - 120ms/epoch - 3ms/step
Epoch 4/200
45/45 - 0s - loss: 3968.8123 - accuracy: 0.2533 - val_loss: 3439.0994 - val_accuracy: 0.2400 - 142ms/epoch - 3ms/step
Epoch 5/200
45/45 - 0s - loss: 2757.3735 - accuracy: 0.2467 - val_loss: 2244.6353 - val_accuracy: 0.2400 - 130ms/epoch - 3ms/step
Epoch 6/200
45/45 - 0s - loss: 1626.8491 - accuracy: 0.2844 - val_loss: 1342.8250 - val_accuracy: 0.4000 - 139ms/epoch - 3ms/step
Epoch 7/200
45/45 - 0s - loss: 1166.8152 - accuracy: 0.4200 - val_loss: 1256.3002 - val_accuracy: 0.4000 - 120ms/epoch - 3ms/step
Epoch 8/200
45/45 - 0s - loss: 1061.6899 - accuracy: 0.4356 - val_loss: 1181.8215 - val_a